In [ ]:
import sys
sys.path.insert(0,'c:/MyDocs/integrated/') # adjust to your setup

%run "catalog_support.py" 

In [ ]:
alldf = pd.read_parquet(os.path.join(hndl.sandbox_dir,'county.parquet'))
alldf.date = pd.to_datetime(alldf.date)
statename = alldf.bgStateName.iloc[0]
countyname = alldf.bgCountyName.iloc[0]
#print(statename, countyname, len(alldf))



In [ ]:
name = f'{countyname.title()} County,<br>{statename.title()}'
showHeader(name, link_up_level=1)

# Where are the fracking locations in this county?
This is not an exhaustive set of wells in this county; it is only those wells for which an operating company submits a chemical disclosure to FracFocus.  In addition, this map omits disclosures for which location information is conflicting, such as the Latitude/Longitude values are outside of the reported county.

In [ ]:
import geopandas as gpd
fn = r"C:\MyDocs\OpenFF\data\non-FF\georef-united-states-of-america-county.geojson"
geojson = gpd.read_file(fn)
geojson['StateName'] = geojson.ste_name.str.lower()
geojson['CountyName'] = geojson.coty_name.str.lower()
geojson = mapping.fix_county_names(geojson)
geojson = geojson[(geojson.StateName==statename)&(geojson.CountyName==countyname)]
shape_flag = len(geojson)==1
# print(shape_flag)

In [ ]:
mapping.create_integrated_point_map(alldf[['DisclosureId','bgLatitude','bgLongitude','APINumber','TBWV','OperatorName',
                                           'year','has_chem','WellName']],
                 aliases=['API Number:','Well Name','Water used:','Operator:','date:','Has chemical records:'],
                 fields=['APINumber','WellName','TBWV','OperatorName','year','has_chem'],
                 include_shape = shape_flag,
                 area_df = geojson
                )

## List of reported disclosures in this county


In [ ]:
def make_disc_link(row):
    return th.getDisclosureLink(row.APINumber,row.DisclosureId,'Open-FF link')
alldf['disc_link'] = alldf.apply(lambda x: make_disc_link(x),axis=1)
def make_FF_link(row):
    return th.getFFLink(row, fmt='short')
alldf['API Number'] = alldf.apply(lambda x: make_FF_link(x),axis=1)

In [ ]:
iShow(alldf[['API Number','disc_link','WellName','TotalBaseWaterVolume','OperatorName','year','has_chem','is_on_DWSHA','is_on_CWA',
             'is_on_PFAS_list']])